In [1]:
from selenium import webdriver
from time import sleep
import calendar
import shutil, os
import zipfile

In [2]:
def fill0(num):
    s = str(num)
    return "0" * (2 - len(s)) + s

In [3]:
exchange_pairs=['AUDJPY', 'AUDNZD', 'AUDUSD', 'CADJPY', 'CHFJPY',
'EURGBP', 'EURJPY', 'EURUSD', 'GBPJPY', 'GBPUSD', 'NZDUSD', 'USDCAD']
years=[str(year) for year in range(2000, 2019)]
months=[[str(calendar.month_name[n]).upper(), fill0(n)] for n in range(1, 13)]

In [4]:
URL0 = "https://truefx.com/dev/data/{YEAR}/{MONTH}-{YEAR}/{PAIR}-{YEAR}-{MON}.zip"
URL1 = "https://truefx.com/dev/data/{YEAR}/{YEAR}-{MON}/{PAIR}-{YEAR}-{MON}.zip"

In [5]:
DIRECTORY = "F:/Dev/Data/truefx/"

In [6]:
class Downloader():
    def __init__(self):
        self.it = self.iterator()
        self.setup()

    def setup(self):
        driver = webdriver.Chrome("F:\Programme\chromedriver.exe")
        driver.get("https://truefx.com/?page=logina")

        el = driver.find_element_by_name("USERNAME")
        el.send_keys("Stephan")

        el = driver.find_element_by_name("PASSWORD")
        el.send_keys("PUT_PASSWORD")

        el = driver.find_element_by_xpath("//input[@value='Login']")
        el.click()

        self.driver = driver

    def download(self):
        URL, name = next(self.it)

        self.driver.get(URL)

        filename = DIRECTORY + name
        shutil.move("C:/Users/Steph/Downloads/" + name, filename)
        
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            zip_ref.extractall(DIRECTORY)

        os.remove(filename)
        
        return_name = name[:-4] + ".csv"
        return return_name

    @staticmethod
    def iterator():
        pair = 'EURUSD'
        for year in years:
            for month, mon in months:
                dict_ = {"YEAR": year, "MONTH": month, "PAIR": pair, "MON": mon}
                if year != "2018":
                    URL = URL0.format_map(dict_)
                else:
                    URL = URL1.format_map(dict_)
                name = "{PAIR}-{YEAR}-{MON}.zip".format_map(dict_)

                print(URL)
                yield URL, name


In [7]:
import datetime
from collections import OrderedDict, deque
import math
import numpy as np
import json, codecs

In [8]:
def parse_time(text):    
    year = int(text[0:4])
    month = int(text[4:6])
    day = int(text[6:8])
    
    hour = int(text[9:11])
    min = int(text[12:14])
    sec = int(text[15:17])
    return datetime.datetime(year, month, day, hour, min, sec)

In [9]:
def map_datetime(dt):
    dt0 = dt
    return dt0.replace(minute=(dt.minute // 15) * 15, second=0)

In [10]:
def get_ohlc(bucket):
    o, c = bucket[0], bucket[-1]
    h = max(bucket, key=lambda a: a[1])
    l = min(bucket, key=lambda a: a[1])
    return o, h, l, c

In [11]:
def calc_z_scores_parameters(cluster):
    cluster0 = np.asarray(cluster)
    mean = np.mean(cluster0, axis=0)
    variance = np.var(cluster0, axis=0)
    return mean, variance

In [12]:
def z_transform(value, mean, variance):
    result = (np.asarray(value) - mean) / variance
    return result.tolist()

In [13]:
def save_data_structure(structure, file):
    json.dump(structure, codecs.open(file, 'w', encoding='utf-8'), sort_keys=True, indent=4)

In [14]:
def reduce_save(name, directory=DIRECTORY):
    datafile = directory + name
    with open(datafile) as f:
        lines = f.readlines()
    
    time = []
    price = []
    for line in lines:
        line_split = line.split(",")
        price.append(0.5*(float(line_split[2]) + float(line_split[3]))) # Ask-Bid-Mitte
        time.append(parse_time(line_split[1]))
        
    buckets = OrderedDict()
    for t, p in zip(time, price):
        printed_time = str(map_datetime(t))
        if printed_time not in buckets:
            buckets[printed_time] = []

        buckets[printed_time].append((t, p))
        
    ohlc = OrderedDict()
    for t, bucket in buckets.items():
        ohlc[t] = get_ohlc(bucket)
        
    closing = list(map(lambda t_v: (t_v[0], t_v[1][3][1]), ohlc.items()))
    
    save_data_structure(closing, DIRECTORY + name[:-4] + ".json")
    
    os.remove(datafile)

In [15]:
downloader = Downloader()

In [16]:
while True:
    try:
        name = downloader.download()
        reduce_save(name)
        break
    except Exception as e:
        print(e)
        break

https://truefx.com/dev/data/2000/JANUARY-2000/EURUSD-2000-01.zip
[Errno 2] No such file or directory: 'C:/Users/Steph/Downloads/EURUSD-2000-01.zip'


In [17]:
a = OrderedDict()
a['1'] = 1
a['2'] = 2

TypeError: <lambda>() missing 1 required positional argument: 'x'